In [1]:
import pandas as pd
from itertools import permutations
from itertools import combinations
from pulp import *
from datetime import datetime

def calculate_distance_haversine(lat1, lon1, lat2, lon2):
    # radius of earth
    from math import radians, cos, sin, asin, sqrt
    r = 3962.173405788

    # convert decimal degrees to radians
    lat1, lon1, lat2, lon2 = [radians(x) for x in [lat1, lon1, lat2, lon2]]
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
    c = 2 * asin(sqrt(a))
    return r * c


# Create the distance matrix between every two locations
def create_distance_matrix(locations):
    distance_matrix = {}
    for ind1, loc1 in locations.iterrows():
        for ind2, loc2 in locations.iterrows():
            dist = calculate_distance_haversine(loc1['Lat'], loc1['Long'], loc2['Lat'], loc2['Long'])
            distance_matrix[(ind1, ind2)] = dist
    return distance_matrix

In [2]:
cust = pd.read_csv('manhattan_customers.csv')
dist = create_distance_matrix(cust)

In [3]:
print(dist[0,0])
print(dist[0,20])

0.0
3.405710719794919


### Worst possible solution: 20 vehicles making 1 stop

In [4]:
tot_dist = 0
for x in range(1,21):
    index = (0, x)
    tot_dist += 2*dist[index]
tot_dist

132.7459446237727

### Model in pulp

In [5]:
destinations = list(range(1,21))
routes_3stop = list(permutations(destinations, 3)) #3420 routes (order matters, but only partially)
routes_2stop = list(combinations(destinations, 2)) #190 routes (order doesn't matter)
routes_1stop = list(permutations(destinations, 1)) #20 routes
routes = routes_3stop + routes_2stop + routes_1stop
routes = [tuple([0]) + x + tuple([0]) for x in routes]

In [6]:
#compute distance of each route
#also, create vector that tells you stops covered by each route
route_info = []
for route in routes:
    tot_dist = 0
    vector = [0]*21
    for x in range(len(route)-1):
        tot_dist += dist[(route[x], route[x+1])]
        vector[route[x]] = 1
    short_route = [str(x) for x in route[1:len(route)-1]]
    name = '-'.join(short_route)
    vector = [str(x) for x in vector[1:21]]
    route_info.append([name, tot_dist, ','.join(vector)])

In [7]:
output = pd.DataFrame(route_info, columns = ['Route', 'Distance', 'Stops'])
output.Distance = round(output.Distance, 6)
output = output.drop_duplicates(subset=['Stops', 'Distance'])

#export the data to CSV
#this csv will be imported in Excel and the problem can be solved using OpenSolver
output.to_csv('HW7_py_output.csv', index = False)

In [8]:
route_info = output.values.tolist()
print('There are a total of {} possible routes'.format(len(route_info))) #expect 3630

There are a total of 3630 possible routes


In [9]:
start=datetime.now()

#Create a list of all the routes
List_Routes = [x[0] for x in route_info]

#Create a dictionary of the distances of the routes
Distances = {}
for x in route_info:
    Distances[x[0]] = x[1]

#Create a list of all the stops
Stops = ['S{}'.format(i) for i in range(1,21)]

#Create a dictionary that accounts for the stops covereb by each route
Stops_Covered = {}
for x in route_info:
    bin_vec = x[2].split(',')
    Stops_Covered[x[0]] = {Stops[i]:int(bin_vec[i]) for i in range(len(Stops))}

# Create the 'prob' variable to contain the problem data
prob = LpProblem("Manhattan_Routing", LpMinimize)

# Create the Variables
routes_vars = {}
for r in List_Routes:
    route_name = '_'.join([str(int(x)+1) for x in r.split('-')])
    routes_vars[r] = LpVariable("{}".format(route_name),lowBound=0,upBound=1,cat=LpBinary)

# The objective function is added to 'prob' first
prob += lpSum([Distances[r]*routes_vars[r] for r in List_Routes]), "Total Distance"

# We can enter the constraints
# Need to stop at all clients, but never more than once
for s in Stops:
    prob += lpSum([routes_vars[r]*Stops_Covered[r][s] for r in List_Routes]) == 1, s
    
# The problem data is written to an .lp file
prob.writeLP("Manhattan_Routing.lp")

# The problem is solved using PuLP's choice of Solver
prob.solve()

# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

# Each of the variables is printed with it's resolved optimum value
counter = 0
for v in prob.variables():
    if v.varValue == 1:
        print('{} is one of the chosen routes'.format(v.name))
        counter += 1
print('The model selected a total of {} routes'.format(counter))
    
# The optimised objective function value is printed to the screen
print("Total Distance = ", value(prob.objective))

dur = datetime.now() - start
dur_f = round(dur.total_seconds(), 2)
print("Total Runtime was {} seconds".format(dur_f))

Status: Optimal
13_17_16 is one of the chosen routes
15_14_18 is one of the chosen routes
2_19 is one of the chosen routes
3_5_6 is one of the chosen routes
4_20_11 is one of the chosen routes
7_12_21 is one of the chosen routes
9_8_10 is one of the chosen routes
The model selected a total of 7 routes
Total Distance =  50.724607
Total Runtime was 2.64 seconds
